In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
import torchvision
from torchvision import models, transforms

import cv2
import json
import glob
import random
import numpy as np
import pandas as pd
import os.path as osp
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from sklearn.preprocessing import normalize


%matplotlib inline

In [2]:
use_pretrained = True
net = models.vgg16(pretrained = use_pretrained)
net

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [3]:
net.classifier[6] = nn.Linear(in_features=4096, out_features=4)

In [4]:
def make_testdatapath_list(rootpath):
    test_path = osp.join(rootpath + "/test_images/")
    test_csv_path = osp.join(rootpath + "/test_images.csv")
    
    test_img_list = list()
    for line in open(test_csv_path):
        if line.strip() == "id":
            continue
        file_id = line.strip()[:14]
        test_img_list.append(test_path + file_id)
        
    return test_img_list

In [5]:
rootpath = "."
test_img_list = make_testdatapath_list(rootpath)
test_img_list[0]

'./test_images/test_0000.jpg'

In [6]:
class TestDataset(data.Dataset):
    
    def __init__(self, test_img_list, transform=None):
        self.test_img_list = test_img_list
        self.transform = transform
        
    def __len__(self):
        return len(self.test_img_list)
    
    def __getitem__(self, index):
        img = self.test_img_list[index]
        
        with open(img, "rb") as f:
            img = Image.open(f)
            img = img.convert("RGB")
            
        if self.transform is not None:
            img = self.transform(img)
        return img

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [8]:
testset = TestDataset(test_img_list, transform=transform)
testloader = DataLoader(testset)

In [9]:
net = net.eval()

In [10]:
batch_iterator = iter(testloader)
img = next(batch_iterator)

In [11]:
len(img)

1

In [14]:
# epoch = 2
load_path = "./weights_fine_tuning.pth"
# # epoch = 30
# load_path = "./weights_fine_tuning2.pth"
load_weights = torch.load(load_path)
net.load_state_dict(load_weights)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [13]:
def test_model(net, dataloader):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Using decide is :", device)
    
    torch.backends.cudnn.benchmark = True
    net.to(device)
    net = net.eval()
    
    pred_list = []
    for data in tqdm(dataloader):
        data = data.to(device)
        data = Variable(data)
        with torch.no_grad():
            output = net(data)
        
        pred_list += [output.data.max(1, keepdim=True)[1]]
    
    return pred_list

In [35]:
pred = test_model(net, testloader)


  0%|          | 2/1651 [00:00<02:40, 10.26it/s]

Using decide is : cuda:0



100%|██████████| 1651/1651 [03:34<00:00,  7.70it/s]


In [40]:
pred
for i in pred:
    print(i)

tensor([[0]], device='cuda:0')
tensor([[3]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([[3]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([[3]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([[3]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([[3]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([[3]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([

In [51]:
a = torch.Tensor([[0], [1], [2]], device="cpu")
b = torch.Tensor([[0]], device="cpu")

a  = a.to("cpu").detach().numpy().copy()
b  = b.to("cpu").detach().numpy().copy()
df = pd.DataFrame(a)
df_temp = pd.DataFrame(b)
df_temp

,0
0,0.0


In [52]:
df = pd.concat([df, df_temp]).reset_index(drop=True)
df

,0
0,0.0
1,1.0
2,2.0
3,0.0


In [55]:
df = pd.DataFrame([])
df

""


In [ ]:
test = []
a_n = a.numpy().reshape(1)
test += a_n
b_n = b.numpy().reshape(1)
test += b_n

In [30]:
test

array([], dtype=float64)

In [24]:
pred_arr = np.array(pred)
pred_arr
pred_arr

array([tensor([[0]], device='cuda:0'), tensor([[3]], device='cuda:0'),
       tensor([[0]], device='cuda:0'), ...,
       tensor([[1]], device='cuda:0'), tensor([[0]], device='cuda:0'),
       tensor([[0]], device='cuda:0')], dtype=object)

In [ ]:
df_test = pd.DataFrame(pred.numpy())
df_test